In [1]:
from sqlalchemy import *
import sys
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date
import plotly.plotly as py
import plotly.graph_objs as go
from axissClass import Axissclass
from axissPrintfigs import printfigs

In [2]:
analysis = Axissclass('1-1-2017','31-07-2017')

In [3]:
df = analysis.proc
pivot = df.pivot_table(index = 'Cluster',values = ['Received','Centre','Reg.No'], aggfunc = {'Received': np.sum,'Reg.No': lambda x: list(x),'Centre':lambda x: list(x)}).reset_index()
pivot['Reg.No'] = pivot.apply(lambda x: len(set(x['Reg.No'])),axis=1)
pivot['Centre'] = pivot.apply(lambda x: set(x['Centre']),axis=1)
pivot['Received'] = pivot.apply(lambda x: np.round(x['Received']*1.0/1e5,0),axis=1)
pivot.sort_values('Received', ascending=False)

,Cluster,Centre,Received,Reg.No
13,South Bangalore,"{HOSUR, ECITY, MSH, CARD}",227.0,4795
0,AECS,"{WHIT, ITPL, AECS}",211.0,3074
9,Noida,"{NOIDA41, YOJNA, NOIDAFH, GZBCAH, IPURAM}",194.0,2244
3,Central,"{BTM, JAYA, HSR, JPN}",182.0,1821
10,North Bangalore,"{SANJ, VIDY, MALL, YELA, RAJA, ASTRYELAold}",152.0,2137
14,South Delhi,"{GK, SDA, VV}",113.0,836
16,West Delhi,"{JANAKPURI, DWARKA, CANTT}",107.0,1344
7,Gur2,"{DLFPH1, GURGAON43}",88.0,831
4,East Bangalore,"{ASTRTPARK, JBN, KAMM, ASTRBLRD, DOML}",84.0,1272
6,Gur1,"{GURGAON56, GURGAON14, GURGAONCAH}",83.0,1093


In [39]:
cl = ['All']
cent = ['All']
df, fig = printfigs (analysis,indicator = 'Received',grplist = ['Year','Month','Classification'],
              regionlist = ['South','North'],clusterlist = cl,centrelist = cent)
py.iplot(fig)

['BLKH', 'CANTT', 'DLFPH1', 'DWARKA', 'FBD14', 'GK', 'GUJ', 'GURGAON14', 'GURGAON43', 'GURGAON56', 'GURGAONCAH', 'GZBCAH', 'IPURAM', 'JANAKPURI', 'MEERA', 'NOIDA41', 'NOIDAFH', 'RAJOURI', 'ROHINI', 'SDA', 'YOJNA', 'AECS', 'BELL', 'BTM', 'CARD', 'DOML', 'ECITY', 'HOSUR', 'HSR', 'ITPL', 'JAYA', 'JBN', 'JPN', 'KAMM', 'LFORD', 'MALL', 'MSH', 'NBEL', 'RAJA', 'SANJ', 'SARJ', 'VIDY', 'WHIT', 'YELA', 'Fortis La Femme', 'ASTRTPARK', 'FORTISBLR', 'ASTRYELA', 'ASTRBLRD', 'VV', 'ASTRYELAold']


In [40]:
df, fig = printfigs (analysis,indicator = 'Reg.No',grplist = ['Year','Month','Classification'],
              regionlist = ['South'],clusterlist = cl,centrelist = cent)
py.iplot(fig)

['AECS', 'BELL', 'BTM', 'CARD', 'DOML', 'ECITY', 'HOSUR', 'HSR', 'ITPL', 'JAYA', 'JBN', 'JPN', 'KAMM', 'LFORD', 'MALL', 'MSH', 'NBEL', 'RAJA', 'SANJ', 'SARJ', 'VIDY', 'WHIT', 'YELA', 'Fortis La Femme', 'ASTRTPARK', 'FORTISBLR', 'ASTRYELA', 'ASTRBLRD', 'ASTRYELAold']


figdatalist = max([len(fig.data[i].get('x')) for i in range(len(fig.data))])
monthlist = [j for j in range (1,figdatalist+1)]
zipped  = [list(zip((fig.data[i].get('x')),(fig.data[i].get('y')),(fig.data[i].get('y')))) for i in range(len(fig.data))]
newzipped = []
for li in zipped:
    if len(li)<len(monthlist):
        monthsli = [i[0] for i in li]
        newli = [(m,0.0,0.0) if m not in monthsli  else (m,li[monthsli.index(m)][1],li[monthsli.index(m)][1]) for m in monthlist ]
        newzipped.append(newli)
    else:
        newzipped.append(li)
def getsum(l1,l2):
    addarray =list(np.array([i[1] for i in l1])+np.array([i[1] for i in l2]))
    addl2 = [(ix+1,ele,l2[ix][2]) for ix,ele in enumerate(addarray)]
    return (addl2)
coords = newzipped
coords = [getsum(coords[ix-1],row) if ix >0 else [(i[0],i[1],i[1]) for i in row] for ix,row in enumerate(newzipped)]
annotations = []
for ix in range(len(coords)):
    for iy in coords[ix]:
        if iy[2]!=0:
            annotations.append(dict(x = iy[0], y = iy[1], text = iy[2], showarrow=False))
monthdf = pd.DataFrame(coords, columns=monthlist).T
monthdf['Total'] = monthdf.apply(lambda x: np.sum([i[2] for i in x]), axis=1)
#for mth,total in zip(monthdf.index,monthdf['Total']):
#    annotations.append(dict(x=mth,y=total+0.25,text = np.round(total,1),showarrow=False))

fig.layout['annotations'] = annotations
fig.layout['barmode'] = 'stack'
fig.layout['title'] = 'Data'



In [91]:
df, fig = printfigs (analysis,indicator = 'Reg.No',grplist = ['Year','Month','Classification'],
              regionlist = ['South'],clusterlist = cl,centrelist = cent)
py.iplot(fig)

['AECS', 'BELL', 'BTM', 'CARD', 'DOML', 'ECITY', 'HOSUR', 'HSR', 'ITPL', 'JAYA', 'JBN', 'JPN', 'KAMM', 'LFORD', 'MALL', 'MSH', 'NBEL', 'RAJA', 'SANJ', 'SARJ', 'VIDY', 'WHIT', 'YELA', 'Fortis La Femme', 'ASTRTPARK', 'FORTISBLR', 'ASTRYELA', 'ASTRBLRD', 'ASTRYELAold']


In [90]:
df = 

,Centre,Reg.No,Reg.Date,Patient Name,Age,Gender,ADP,Date,Tooth No,Diagnosis,...,Cluster,Cluster Head,Hospital,Unnamed: 0_y,Classification,PatientType,AgeBand,Month,Year,DayofWeek
0,MOHALIFH,10,04-07-2017,ANJANA DEVI,41.0,F,NO,04-07-2017,FM,None,...,Punjab,Dev,Yes,26.0,Not Classified,New,Elder 40-80,7,2017,1
1,SDA,11100062,13-10-2000,MALVIKA JAIN,18.0,F,NO,14-04-2017,26,Caries,...,South Delhi,Aditi,No,3.0,Primary,Old,Youth 13-25,4,2017,4
2,SDA,11100062,13-10-2000,MALVIKA JAIN,18.0,F,NO,14-04-2017,FM,Gingivitis,...,South Delhi,Aditi,No,21.0,Primary,Old,Youth 13-25,4,2017,4
3,SDA,11100062,13-10-2000,MALVIKA JAIN,18.0,F,NO,14-04-2017,46,None,...,South Delhi,Aditi,No,12.0,Secondary,Old,Youth 13-25,4,2017,4
4,SDA,11100199,04-06-2001,JUGAL KISHORE,51.0,M,NO,29-04-2017,FM,None,...,South Delhi,Aditi,No,18.0,Primary,Old,Elder 40-80,4,2017,5
5,SDA,11100199,04-06-2001,JUGAL KISHORE,51.0,M,NO,29-04-2017,FM,None,...,South Delhi,Aditi,No,2.0,Primary,Old,Elder 40-80,4,2017,5
6,SDA,11100234,03-05-2001,MARRY MATTHAI,68.0,F,NO,14-06-2017,27,Caries,...,South Delhi,Aditi,No,3.0,Primary,Old,Elder 40-80,6,2017,3
7,SDA,11100234,03-05-2001,MARRY MATTHAI,68.0,F,NO,28-06-2017,46,Crown dislodged,...,South Delhi,Aditi,No,16.0,Secondary,Old,Elder 40-80,6,2017,2
8,SDA,11100272,28-12-2000,PREM SOFTA,NaN,None,None,29-03-2017,14,Edentulous,...,South Delhi,Aditi,No,5.0,Tertiary,Old,Check,3,2017,4
9,SDA,11100272,28-12-2000,PREM SOFTA,65.0,M,NO,29-03-2017,24,Functional Problem,...,South Delhi,Aditi,No,16.0,Secondary,Old,Elder 40-80,4,2017,2


In [93]:
df = analysis.proc


In [117]:
pt2 = df.pivot_table(index = ['Region','AgeBand'], values = ['Reg.No','Received'], columns = ['Month'], aggfunc = {'Reg.No': lambda x: len(list(set(x))),'Received': np.sum}).reset_index()

In [118]:
pt2.to_csv('pt2.csv')

In [120]:
df['AgeBand'].unique()

array(['Elder 40-80', 'Youth 13-25', 'Check', 'Mid 26-40', 'Kids <=12',
       'Old >=81'], dtype=object)

In [133]:
d1 = df[df['AgeBand']=='Elder 40-80']
print (d1['Received'].sum()/1e5)
d1[(d1['Age']>=40)&(d1['Age']<=55)]['Received'].sum()/1e5

477.65433


233.78583

In [134]:
df[df['Month'].isin(range(4,8))]['Received'].sum()/1e5

1024.19085